# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [21]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [23]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,11.01,73,100,0.89,NZ,1722820516
1,1,dudinka,69.4058,86.1778,10.22,56,0,9.22,RU,1722820310
2,2,udachny,66.4167,112.4000,19.88,44,100,4.81,RU,1722820455
3,3,albany,42.6001,-73.9662,24.24,77,0,0.45,US,1722820066
4,4,wenling,28.3669,121.3600,29.63,76,53,2.17,CN,1722820394


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [24]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 520,
    size = "Humidity",
    scale = 1,
    color = "City")
# Display the map
humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
ideal = city_data_df.loc[(city_data_df["Max Temp"] < 32) & (city_data_df["Max Temp"] > 25) & (city_data_df["Cloudiness"] < 45) & (city_data_df["Wind Speed"] < 15) & (city_data_df["Humidity"] < 75)]

# Drop any rows with null values
ideal = ideal.dropna()

# Display sample data
ideal.head(20)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,saipan,15.1355,145.7010,31.39,66,40,6.69,MP,1722820533
59,59,saynshand,44.8824,110.1163,25.39,32,0,3.80,MN,1722820583
61,61,tayu,-6.5397,111.0518,26.38,67,8,3.93,ID,1722820586
65,65,trinidad,-14.8333,-64.9000,26.20,54,30,0.61,BO,1722820590
66,66,santa ana de yacuma,-13.7500,-65.4333,28.71,42,8,1.90,BO,1722820591
90,90,idri,27.5000,13.2667,28.52,17,0,1.69,LY,1722820618
91,91,kingston,17.9970,-76.7936,30.53,74,20,6.69,JM,1722820155
101,101,haiku-pauwela,20.9219,-156.3051,29.03,62,20,12.86,US,1722820631
114,114,yeraliyev,43.2000,51.6500,27.63,39,0,3.85,KZ,1722820646
118,118,martapura,-3.4167,114.8500,27.05,74,0,1.54,ID,1722820651


### Step 3: Create a new DataFrame called `hotel_df`.

In [26]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
16,saipan,MP,15.1355,145.7010,66,None
59,saynshand,MN,44.8824,110.1163,32,None
61,tayu,ID,-6.5397,111.0518,67,None
65,trinidad,BO,-14.8333,-64.9000,54,None
66,santa ana de yacuma,BO,-13.7500,-65.4333,42,None
90,idri,LY,27.5000,13.2667,17,None
91,kingston,JM,17.9970,-76.7936,74,None
101,haiku-pauwela,US,20.9219,-156.3051,62,None
114,yeraliyev,KZ,43.2000,51.6500,39,None
118,martapura,ID,-3.4167,114.8500,74,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saipan - nearest hotel: Chalan Kanoa Beach Hotel
saynshand - nearest hotel: No hotel found
tayu - nearest hotel: No hotel found
trinidad - nearest hotel: Hotel Palace
santa ana de yacuma - nearest hotel: No hotel found
idri - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite Hotel
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
yeraliyev - nearest hotel: Kuryk
martapura - nearest hotel: Q-Hotel Grand Dafam
smara - nearest hotel: Hôtel Amine فندق الأمين
avarua - nearest hotel: Paradise Inn
lahaina - nearest hotel: No hotel found
sitka - nearest hotel: Aspen Suites Hotel
balkanabat - nearest hotel: Nebitçi
pimenta bueno - nearest hotel: Cristalina palace hotel
hamilton - nearest hotel: North Vista Manor
al jawf - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
mamuju - nearest hotel: Hotel d'Maleo
metlili chaamba - nearest hotel: No hotel found
papao - nearest hotel: Hiti Moana Villa Lodge
nova olimpia - nearest hot

,City,Country,Lat,Lng,Humidity,Hotel Name
16,saipan,MP,15.1355,145.7010,66,Chalan Kanoa Beach Hotel
59,saynshand,MN,44.8824,110.1163,32,No hotel found
61,tayu,ID,-6.5397,111.0518,67,No hotel found
65,trinidad,BO,-14.8333,-64.9000,54,Hotel Palace
66,santa ana de yacuma,BO,-13.7500,-65.4333,42,No hotel found
90,idri,LY,27.5000,13.2667,17,No hotel found
91,kingston,JM,17.9970,-76.7936,74,Aphrodite Hotel
101,haiku-pauwela,US,20.9219,-156.3051,62,Inn At Mama's Fish House
114,yeraliyev,KZ,43.2000,51.6500,39,Kuryk
118,martapura,ID,-3.4167,114.8500,74,Q-Hotel Grand Dafam


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 520,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Lat","Lng","City","Humidity","Hotel Name","Country"] 
)

# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)